In [1]:
import dill

import numpy as np
import squigglepy as sq

from squigglepy.numbers import K, M, B
from squigglepy import bayes
from tqdm import tqdm
from pprint import pprint
from copy import deepcopy

exec(open('utils.py').read())
print('Loaded')

Loaded


In [2]:
with open('caches/variables.dill', 'rb') as f:
    VARS = dill.load(f)
print('loaded default variables from cache!') # Default variables are defined in "(4) XRisk Model.ipynb"
# TODO: can do sensitivity analysis or VOI analysis over all the variables

loaded default variables from cache!


In [3]:
VARS['tai_years'] = np.array(VARS['tai_years'])
VARS

{'RUNS': 50000,
 'CURRENT_YEAR': 2023,
 'MAX_YEAR': 2123,
 'p_make_agent_tai': 0.9,
 'p_tai_aligned_by_default': 0.3,
 'p_alignment_solved': <function __main__.p_alignment_solved(war, year, first_attempt=True, verbose=False)>,
 'p_alignment_deployment_safety_and_coordination': <function __main__.p_alignment_deployment_safety_and_coordination(war, year, variables, first_attempt=True, verbose=False)>,
 'p_subtle_alignment_solved': 0.85,
 'p_subtle_alignment_solved_if_aligned_by_default': 0.4,
 'p_tai_intentional_misuse': <function __main__.p_tai_intentional_misuse(war)>,
 'p_full_tai_misalignment_averted': 0.15,
 'p_tai_misalignment_averting_is_catastrophic': 0.4,
 'p_full_tai_misalignment_averted_means_abandoned_tai': 0.7,
 'p_tai_xrisk_is_extinction': 0.4,
 'p_tai_singleton_is_catastrophic': 0.8,
 'p_russia_uses_nuke': <function __main__.p_russia_uses_nuke(peace, year, variables)>,
 'p_nk_uses_nuke': 0.001,
 'p_china_invades_taiwan': <function __main__.p_china_invades_taiwan(peace, yea

In [4]:
years = range(VARS['CURRENT_YEAR'], VARS['MAX_YEAR'])

exec(open('modules/tai_risk.py').read())
print('Loaded TAI scenarios module')

exec(open('modules/nuclear.py').read())
print('Loaded nuclear scenarios module')

exec(open('modules/great_power_war.py').read())
print('Loaded great power war scenarios module')

exec(open('modules/bio.py').read())
print('Loaded bio scenarios module')

exec(open('modules/nano.py').read())
print('Loaded nano scenarios module')

exec(open('modules/supervolcano.py').read())
print('Loaded supervolcano module')

exec(open('modules/unknown_unknown.py').read())
print('Loaded unknown unknown scenarios module')

exec(open('modules/double_dip_catastrophe.py').read())
print('Loaded double dip catastrophe module')

exec(open('modules/tai_timelines.py').read())
print('Loaded TAI timelines module')

tai_years = bayes.bayesnet(load_cache_file='caches/tai_years', verbose=True)
print('loaded TAI variables from cache')

Loaded TAI scenarios module
Loaded nuclear scenarios module
Loaded great power war scenarios module
Loaded bio scenarios module
Loaded nano scenarios module
Loaded supervolcano module
Loaded unknown unknown scenarios module
Loaded double dip catastrophe module
Loaded TAI timelines module
Loading from cache file (`caches/tai_years.sqcache`)...
...Loaded
Caching in-memory...
...Cached!
...Reducing
...Reduced!
...All done!
loaded TAI variables from cache


In [5]:
# TODO: Variation on these inputs?
human_population = 8*B # TODO: Animals? Chance present is net negative?
qaly_per_person = 40 # TODO: Improvements in health over time?
VARS['total_present_value'] = human_population * qaly_per_person
print('Total value at present: {} QALY'.format(numerize(VARS['total_present_value'])))

births_per_year = 100*M # TODO: Digital minds? Population decline?
qaly_per_birth = 60 # TODO: Improvements in health over time?
VARS['total_additional_value_per_year'] = births_per_year * qaly_per_birth

VARS['years_to_consider'] = 100  # TODO: Expand somehow to include more years?
total_additional_value = VARS['total_additional_value_per_year'] * VARS['years_to_consider']
print('Total additional value over 100 years: {} QALY'.format(numerize(total_additional_value)))

total_value = VARS['total_present_value'] + total_additional_value
print('Total value of future: {} QALY'.format(numerize(total_value)))

Total value at present: 320 billion QALY
Total additional value over 100 years: 600 billion QALY
Total value of future: 920 billion QALY


In [6]:
exec(open('modules/world_state_value.py').read())
print('Loaded world state valuation module')

Loaded world state valuation module


In [7]:
%%time

exec(open('modules/define_event.py').read())
print('Model loaded')

# TODO: Reduce amount of information in cache file (only need final year) to improve load speed
collectors = bayes.bayesnet(define_event,
                            find=lambda e: e['collectors'][VARS['MAX_YEAR'] - 1],
                            load_cache_file='caches/future_assessment_model_cache_short',
                            reload_cache=False,
                            raw=True,
                            verbose=True,
                            cores=1,
                            n=VARS['RUNS'])
collectors[0]

Model loaded
Loading from cache file (`caches/future_assessment_model_cache_short.sqcache`)...
...Loaded
Caching in-memory...
...Cached!
...Finding
...Found!
...All done!
CPU times: user 29 s, sys: 6.81 s, total: 35.8 s
Wall time: 39.7 s


{'category': 'boring',
 'tai': False,
 'tai_year': None,
 'tai_type': None,
 'nano': False,
 'wars': [{'belligerents': 'US/Russia',
   'start_year': 2089,
   'end_year': 2091,
   'war_length': 2}],
 'war': False,
 'war_start_year': 2089,
 'war_end_year': 2091,
 'russia_nuke_first': False,
 'china_nuke_first': False,
 'war_belligerents': None,
 'peace_until': 2181,
 'engineered_pathogen': True,
 'natural_pathogen': True,
 'lab_leak': True,
 'state_bioweapon': False,
 'nonstate_bioweapon': False,
 'averted_misalignment': False,
 'nuclear_weapon_used': False,
 'catastrophe': ['engineered_pathogen'],
 'recent_catastrophe_year': 2025,
 'terminate': False,
 'final_year': None,
 'double_catastrophe_xrisk': None}

In [8]:
value_of_future = [value_of_world_state(world_state=c, variables=VARS) for c in tqdm(collectors)]

print('EV of future: {} QALY'.format(numerize(np.mean(value_of_future))))
print('-')
pprint(sq.get_log_percentiles(value_of_future))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 136271.70it/s]

EV of future: 626 billion QALY
-
{1: '3.6e+11',
 5: '3.9e+11',
 10: '4.0e+11',
 20: '4.3e+11',
 30: '4.5e+11',
 40: '4.8e+11',
 50: '5.2e+11',
 60: '5.9e+11',
 70: '9.2e+11',
 80: '9.2e+11',
 90: '9.2e+11',
 95: '9.2e+11',
 99: '9.2e+11'}


In [9]:
%%time
alt_variables = deepcopy(VARS)
alt_variables['tai_years'] = [t + 5 for t in alt_variables['tai_years']] # Uniformly and universally delay TAI by 5 years with 100% success

# TODO: Be able to declare changes in variables for only particular years

print('Running intervention model...')
alt_define_event_lambda = lambda: define_event(alt_variables, verbosity=0)
alt_collectors = bayes.bayesnet(alt_define_event_lambda,
                                find=lambda e: e['collectors'][VARS['MAX_YEAR'] - 1],
                                raw=True,
                                verbose=True,
                                cores=5,
                                n=VARS['RUNS'])
print('Ready')

Running intervention model...
Generating Bayes net with 5 cores...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [07:57<00:00, 104.74it/s]


Shuffling data...
Waiting for other cores...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:48<00:00,  9.76s/it]

...Collected!
Caching in-memory...
...Cached!
...Finding
...Found!
...All done!
Ready
CPU times: user 8min 8s, sys: 24.9 s, total: 8min 33s
Wall time: 8min 48s


In [10]:
print('Calculating value...')
value_of_alt_future = [value_of_world_state(world_state=c, variables=alt_variables) for c in tqdm(alt_collectors)]
print('Ready')

Calculating value...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 648887.35it/s]

Ready


In [12]:
print('-')
print('EV of default future: {} QALY'.format(numerize(np.mean(value_of_future))))
print('EV of alt future: {} QALY'.format(numerize(np.mean(value_of_alt_future))))
print('-')
print('Diff / Value of intervention: Alt future is {} QALY relative to default'.format(numerize(np.mean(value_of_alt_future) - np.mean(value_of_future))))
print('-')
print('Distribution of default future')
pprint(sq.get_log_percentiles(value_of_future))
print('-')
print('Distribution of alt future')
pprint(sq.get_log_percentiles(value_of_alt_future))


-
EV of default future: 626 billion QALY
EV of alt future: 646 billion QALY
-
Diff / Value of intervention: Alt future is 21 billion QALY relative to default
-
Distribution of default future
{1: '3.6e+11',
 5: '3.9e+11',
 10: '4.0e+11',
 20: '4.3e+11',
 30: '4.5e+11',
 40: '4.8e+11',
 50: '5.2e+11',
 60: '5.9e+11',
 70: '9.2e+11',
 80: '9.2e+11',
 90: '9.2e+11',
 95: '9.2e+11',
 99: '9.2e+11'}
-
Distribution of alt future
{1: '3.9e+11',
 5: '4.2e+11',
 10: '4.3e+11',
 20: '4.6e+11',
 30: '4.8e+11',
 40: '5.1e+11',
 50: '5.5e+11',
 60: '6.2e+11',
 70: '9.2e+11',
 80: '9.2e+11',
 90: '9.2e+11',
 95: '9.2e+11',
 99: '9.2e+11'}
